In [1]:
import os
import gym
import gym_donkeycar
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2 as cv
import time
import pickle
import birds_eye_vector_space
import random
from pandas import Series, DataFrame
from collections import deque

#from keras.layers import Dense
#from tensorflow.keras.optimizers import Adam
#from keras.initializers import normal, identity
#from keras.models import model_from_json
#from keras.models import Sequential
#from keras.layers.core import Dense, Dropout, Activation, Flatten
#from keras.layers.convolutional import Convolution2D, MaxPooling2D

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.client import device_lib
#from keras import backend as K
#import tensorflow.keras.backend as K

C:\Users\studwilksa2535\AppData\Roaming\Python\Python38\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:18: DeprecationWarning: Call to deprecated create function FileDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  DESCRIPTOR = _descriptor.FileDescriptor(
C:\Users\studwilksa2535\AppData\Roaming\Python\Python38\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:36: DeprecationWarning: Call to deprecated create function FieldDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  _descriptor.FieldDescriptor(
C:\Users\studwilksa2535\AppData\Roaming\Python\Python38\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:29: DeprecationWarning: Call to deprecated create function Descriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find d

In [2]:
# use gpu
#K.tensorflow_backend._get_available_gpus()
#sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
#K.tensorflow_backend._get_available_gpus()

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  2


In [4]:
tf.test.is_built_with_cuda()

True

In [5]:
tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [6]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]

In [7]:
# run on the second GPU
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
#os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [8]:
len(tf.config.list_physical_devices('GPU'))
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 15626152661889525869
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 9401860096
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 6091412644311600745
 physical_device_desc: "device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:49:00.0, compute capability: 7.5"
 xla_global_id: 416903419,
 name: "/device:GPU:1"
 device_type: "GPU"
 memory_limit: 6271991808
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 7161733880544342748
 physical_device_desc: "device: 1, name: GeForce RTX 2070 SUPER, pci bus id: 0000:21:00.0, compute capability: 7.5"
 xla_global_id: 2144165316]

In [9]:
#mirrored_strategy = tf.distribute.MirroredStrategy()

In [10]:
#sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [11]:
# Supress scientific notation like: e*+03
np.set_printoptions(suppress=True)

In [12]:
ANCHOR_POINTS = 8
ROAD_ROI = np.array([(120,90),(200,90),(0,200),(320,200)],dtype='float32')
WARPED_IMAGE_SHAPE = np.array([[10,320],[0,0],[200,0],[200,310]],np.int32)            # NEW IMAGE Shape after Warping !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
WARPED_IMAGE_HEIGHT = 320
WARPED_IMAGE_WIDTH = 200
NUMBER_OF_DEPTH_LAYERS = 50

VECTOR_SPACE_IMAGE_ROWS = 50
VECTOR_SPACE_IMAGE_COLUMNS = 100
VECTOR_SPACE_IMAGE_CHANNELS = 4 # 4*(50,100) stacked frames 

In [13]:
# function to save the plots
def save_plot(data, name, episode):
    plt.figure(figsize=(8,5), frameon=True)
    #plt.plot([episode for episode in range(len(data))], data)
    plt.plot([ep for ep in range(episode)], data)
    plt.xlabel('Episodes')
    plt.ylabel(name)
    figplot = '%s-episode-%s.png' % (name,episode)
    #figplot = '{}-episode-{}.png'.format(name,episode)
    #location = '{}/saves/torch/{}'.format(os.getcwd(),figplot)
    #location = '%s/collected_data/plots/%s' % (os.getcwd(),figplot)
    location = 'C:\\Users\\studwilksa2535\\Desktop\\DonkeyCarAI\\collected_data\\plots\\%s' % figplot 
    plt.savefig(location, transparent=False)
    plt.close()

In [14]:
def save_data_as_dataframe(episode_number, reward, loss, measured_time, max_cte, average_cte, last_lap_time):
    episodes = [i for i in range(1, episode_number+1)]
    collected_data = {'episode': episodes, 'reward': reward, 'loss':loss, 'time': measured_time, 'max_cte': max_cte, 'average_cte': average_cte, 'lap_time': last_lap_time}
    df_data = DataFrame.from_dict(collected_data).set_index('episode')
    
    df_name = 'data-episode-%s.pkl' % episode
    #location = '%s/collected_data/raw_data/%s' % (os.getcwd(),df_name)
    location = 'C:\\Users\\studwilksa2535\\Desktop\\DonkeyCarAI\\collected_data\\raw_data\\%s' % df_name
    df_data.to_pickle(location)

In [15]:
vector_space = birds_eye_vector_space.Vector_space(ANCHOR_POINTS, ROAD_ROI, WARPED_IMAGE_SHAPE, WARPED_IMAGE_HEIGHT, WARPED_IMAGE_WIDTH, NUMBER_OF_DEPTH_LAYERS)

vector space is initialized
loading camera parameters


In [16]:
os.getcwd()

'C:\\Users\\studwilksa2535\\Desktop\\DonkeyCarAI\\Master-Thesis-Development-of-a-Deep-RL-Model-for-simulated-Driving-2D-Vector-Space\\docker\\src'

In [17]:
class DQNAgent:
    
    def __init__(self, state_size, action_size):
        self.t = 0
        self.max_Q = 0
        self.train = True
        # Set to True to train on images with segmented lane lines
        self.lane_detection = False
        
        # Huber loss
        self.huber_loss = tf.keras.losses.Huber(reduction=tf.keras.losses.Reduction.SUM)
        
        # Get size of state and action
        self.state_size = state_size
        self.action_size = action_size
        
        # CNN Input
        #self.input_shape = (VECTOR_SPACE_IMAGE_CHANNELS, VECTOR_SPACE_IMAGE_ROWS, VECTOR_SPACE_IMAGE_COLUMNS, 4)   # == 4 * (50, 100, 1); keras: (1,50,100,4)
        self.input_shape = (1, VECTOR_SPACE_IMAGE_ROWS, VECTOR_SPACE_IMAGE_COLUMNS, VECTOR_SPACE_IMAGE_CHANNELS)   # == 4 * (50, 100, 1); keras: (1,50,100,4)
        
        # These are hyper parameters for the DQN
        self.discount_factor = 0.99
        self.learning_rate = 1e-4
        if (self.train):
            self.epsilon = 1.0
            self.initial_epsilon = 1.0
        else:
            self.epsilon = 1e-6
            self.initial_epsilon = 1e-6
        self.epsilon_min = 0.02
        self.batch_size = 512
        self.train_start = 100
        self.explore = 10000
        
        # Create replay memory using deque
        self.memory = deque(maxlen=10000)
        
        # Create main model and target model                # Double DQN !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        self.model = self.build_model()
        self.target_model = self.build_model()

        # Copy the model to target model
        # --> initialize the target model so that the parameters of model & target model to be same
        self.update_target_model()
        
    def build_model_old(self):
        print('delte this')
        #model = Sequential()
        #model.add(Convolution2D(32, 8, 8, subsample=(4, 4), border_mode='same',input_shape=(VECTOR_SPACE_IMAGE_ROWS,VECTOR_SPACE_IMAGE_COLUMNS,VECTOR_SPACE_IMAGE_CHANNELS)))  #80*80*4
        #model.add(Activation('relu'))
        #model.add(Convolution2D(64, 4, 4, subsample=(2, 2), border_mode='same'))
        #model.add(Activation('relu'))
        #model.add(Convolution2D(64, 3, 3, subsample=(1, 1), border_mode='same'))
        #model.add(Activation('relu'))
        #model.add(Flatten())
        #model.add(Dense(512))
        #model.add(Activation('relu'))

        ## 15 categorical bins for Steering angles
        #model.add(Dense(15, activation="linear")) 

        #adam = Adam(lr=self.learning_rate)
        #model.compile(loss='mse',optimizer=adam)

        #return model

    def build_model(self):
        model = keras.Sequential([
            layers.Conv2D(filters=32, strides=(4, 4),kernel_size=8, padding='same', activation='relu', input_shape=(self.input_shape[1:])),
            layers.Conv2D(filters=64, strides=(2, 2),kernel_size=4, padding='same', activation='relu'),
            layers.Conv2D(filters=64, strides=(1, 1),kernel_size=3, padding='same', activation='relu'),
            layers.Flatten(),
            layers.Dense(units=512,activation='relu'),
            layers.Dense(units=512,activation='relu'), # TODO ADDED THIS LAYER FOR TESTING
            layers.Dense(units=15,activation='linear'),
        ])
        #optimizer = tf.keras.optimizers.Adam(learning_rate=self.learning_rate)
        optimizer = tf.keras.optimizers.Adam()
        #optimizer = tf.keras.optimizers.SGD()
        model.compile(loss='mse', optimizer=optimizer)
        #model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=optimizer)
        return model
    
    def calculate_loss(self, y_true, y_pred):    
        y_pred = tf.convert_to_tensor_v2(y_pred)
        y_true = tf.cast(y_true, y_pred.dtype)
        return tf.reduce_mean(math_ops.square(y_pred - y_true), axis=-1)
    
    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

    # Get action from model using epsilon-greedy policy
    def get_action(self, s_t):
        if np.random.rand() <= self.epsilon:
            #print("Return Random Value")
            #return random.randrange(self.action_size)
            return np.random.uniform(-1,1)
        else:
            #print("Return Max Q Prediction")
            q_value = self.model.predict(s_t)
            # Convert q array to steering value
            return linear_unbin(q_value[0])

    def replay_memory(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        #if self.epsilon > self.epsilon_min:
        #    #self.epsilon *= self.epsilon_decay
        #    self.epsilon -= (self.initial_epsilon - self.epsilon_min) / self.explore

    def train_replay(self):
        if len(self.memory) < self.train_start:
            return
        
        batch_size = min(self.batch_size, len(self.memory))
        minibatch = random.sample(self.memory, batch_size)

        state_t, action_t, reward_t, state_t1, terminal = zip(*minibatch)
        #print('Starting training\nstate_t: %s\naction_t: %s\nreward_t: %s\nstate_1: %s\nterminal: %s' % (len(state_t), len(action_t), len(reward_t), len(state_t1), len(terminal)))
        state_t = np.concatenate(state_t)
        state_t1 = np.concatenate(state_t1)
        #print('Concat:\nstate: %s\nstate_t1: %s' % (state_t, state_t1))
        targets = self.model.predict(state_t)
        self.max_Q = np.max(targets[0])
        target_val = self.model.predict(state_t1)
        target_val_ = self.target_model.predict(state_t1)
        for i in range(batch_size):
            if terminal[i]:
                targets[i][action_t[i]] = reward_t[i]
            else:
                a = np.argmax(target_val[i])
                targets[i][action_t[i]] = reward_t[i] + self.discount_factor * (target_val_[i][a])

        
        # train on the current batch and return a dictionary with the loss and so on
        #metrics = self.model.train_on_batch(x=state_t, y=targets, reset_metrics=True, return_dict=True)           
        metrics = self.model.train_on_batch(x=state_t, y=targets, reset_metrics=True, return_dict=True) #reset Metrix different
        return metrics['loss']
        
    def load_model(self, name):
        self.model.load_weights(name)

    # Save the model which is under training
    def save_model(self, name):
        self.model.save_weights(name)

In [18]:
def linear_bin(a):
    """
    Convert a value to a categorical array.

    Parameters
    ----------
    a : int or float
        A value between -1 and 1

    Returns
    -------
    list of int
        A list of length 15 with one item set to 1, which represents the linear value, and all other items set to 0.
    """
    a = a + 1
    b = round(a / (2 / 14))
    arr = np.zeros(15)
    arr[int(b)] = 1
    return arr

In [19]:
def linear_unbin(arr):
    """
    Convert a categorical array to value.

    See Also
    --------
    linear_bin
    """
    if not len(arr) == 15:
        raise ValueError('Illegal array length, must be 15')
    b = np.argmax(arr)
    a = b * (2 / 14) - 1
    return a

## Main training loop

In [20]:
#%% SET UP ENVIRONMENT
# Normal one 
#os.environ['DONKEY_SIM_PATH'] = "/home/zamy/masterthesis/DonkeySimLinux/donkey_sim.x86_64"
os.environ['DONKEY_SIM_PATH'] = "C:\\Users\\studwilksa2535\\Desktop\\DonkeyCarAI\\test\\DonkeySimWin\\donkey_sim.exe"
os.environ['DONKEY_SIM_PORT'] = str(9091)
os.environ['DONKEY_SIM_HEADLESS'] = str(0) # "1" is headless

CAMERA_CONF = {'cam_config':{'img_w': '320',
                        'img_h': '200',
                        'img_d': '1',   # 3 for colored Tensor image
                        'img_enc': 'PNG', 
                        'fov': '90', 
                        'fish_eye_x': '0.0', 
                        'fish_eye_y': '0.0', 
                        'offset_x': '0.0', 
                        'offset_y': '0.0', 
                        'offset_z': '0.0', 
                        'rot_x': '0'}}
# Other tracks
#env = gym.make("donkey-generated-track-v0",conf=CAMERA_CONF)

# TRACKS TRACKS - TKarmer Tracks
#env = gym.make("donkey-generated-track-v0",conf=CAMERA_CONF) #,conf=config
# Random track, but the reward is kinda not working, IF THE REWARD WORKS IT IS KINDA RANDOM !!, this maybe debends on the environment
env = gym.make("donkey-generated-roads-v0",conf=CAMERA_CONF) #,conf=config

# Mini Monaco Track
#env = gym.make("donkey-minimonaco-track-v0",conf=CAMERA_CONF) #,conf=config
#env = gym.make("donkey-generated-track-v0") #,conf=config
#env = gym.wrappers.ResizeObservation(env,(200,320))

## DELETE LATER
#env.frameskip = 1
#gym.wrappers.max

# Create DQN Model
# Get size of state and action from environment
state_size = (VECTOR_SPACE_IMAGE_ROWS, VECTOR_SPACE_IMAGE_COLUMNS, VECTOR_SPACE_IMAGE_CHANNELS)
action_size = 15 # Steering and Throttle
throttle = 0.15 # Set the throttle as a constant value
agent = DQNAgent(state_size, action_size)


# Arrays for data collection / exploratory data analysis
average_cte = []
average_speed = []
measured_time = []
average_rewards = []
average_loss = []
collected_max_cte = []
collected_lap_time = []

# when a model and the data should be saved
save_state = 200

for episode in range(1,7001):
    # Resetting the environment and preprocessing the first image
    observation = env.reset()
    # Graykonvertion of the observation image
    observation = cv.cvtColor(observation, cv.COLOR_BGR2GRAY)
    observation = vector_space.image_preprocessing(observation)
    # frame stacking (4 times)
    obv_stack = np.stack((observation, observation, observation, observation), axis= 2)
    # reshaping for keras
    obv_stack = obv_stack.reshape(1, obv_stack.shape[0], obv_stack.shape[1], obv_stack.shape[2])
    
    #action = np.array([0,0.10]) # drive straight with small speed
    #action = np.array([1,0.1]) # drive straight with small speed
    
    # summed up values for data collection
    # cumulative values for data collection
    total_cte = float(0)
    total_speed = float(0)
    total_reward = float(0)
    total_time = time.time()
    total_loss = float(0)
    max_cte = int(0)
    lap_time = int(0)
        
    # counting the amount of frames per episode
    frames = int(0)
    
    # boolean that describes if the env is done with this episode
    done = False
    while not done:
        # incrementing the amount of frames per episode
        frames += 1
        
        # Making a prediction for the current state
        # and getting the information form the next step
        steering = agent.get_action(obv_stack)
        action = [steering, throttle]
        next_observation, reward, done, info = env.step(action)
        
        current_cte = abs(info['cte'])
        # punish and reward for a low or high 'CTE' with the logarithmic function
        if current_cte < .25:
            reward += abs(np.log(.10)) * 8  # 4 before
        elif current_cte >= .25 and current_cte < .5:
            reward += current_cte * 2
        elif current_cte >= .5 and current_cte < 1: 
            reward += abs(np.log(current_cte))
        elif current_cte >= 1 and current_cte < 2: 
            reward -= abs(current_cte) * 2
        elif current_cte >= 2 and current_cte < 3: 
            reward -= abs(current_cte) * 3
        elif current_cte >= 3 and current_cte < 4: 
            reward -= abs(current_cte) * 4
        elif current_cte >=4:
            reward -= abs(current_cte) * 5
                    
        # if the car leaves the track punish it
        if done:
            reward -= 100
        # Check if the CTE of the environment is bugged
        if episode == 1:
            #print('CTE: %s\tReward: %s' % (info['cte'], reward)
            print('REWARD: %s\tCTE: %s' % (reward, current_cte))
        
        # Graykonvertion of the observation image
        next_observation = cv.cvtColor(next_observation, cv.COLOR_BGR2GRAY)
        
        # preprocessing the new observation
        next_observation_show = vector_space.image_preprocessing(next_observation)
        # reshaping for keras
        #next_observation = next_observation_show.reshape(1, next_observation_show.shape[0], next_observation_show.shape[1], 1)
        # appending to the observation stack
        #obv_stack = np.append(next_observation, obv_stack[:, :, :, :3], axis=3)
        
        obv_stack_t1 = np.stack([next_observation_show, obv_stack[0,:,:,0],obv_stack[0,:,:,1],obv_stack[0,:,:,2]], axis=2)
        obv_stack_t1 = obv_stack_t1.reshape(1, obv_stack_t1.shape[0], obv_stack_t1.shape[1], obv_stack_t1.shape[2])
                
        # saving the sample <s, a, r, s'> to the replay memory
        agent.replay_memory(obv_stack, np.argmax(linear_bin(steering)), reward, obv_stack_t1, done)        
        
        # training the DDQN, if training is enabled
        #if agent.train:
        #    loss = agent.train_replay()
        #    if loss != None:
        #        total_loss += loss
        #        #print('loss: %s ' % (total_loss / frames))
        
        # overwriting the stack and incrementing the time/frame counter
        obv_stack = obv_stack_t1
        agent.t += 1       
        
        # adding up the collected data
        current_cte = info['cte'] 
        total_cte += current_cte
        total_speed += info['speed']
        total_reward += reward
        # update if there is any change in incrementation 
        if max_cte < current_cte:
            max_cte = current_cte
        if lap_time < info['last_lap_time']:
            lap_time = info['last_lap_time']
            
        cv.imshow('vec img', next_observation_show)
        
        # stop the training if the car passes 120 seconds,
        # so there is no infinit learning
        current_time = abs(total_time - time.time())
        if current_time >= 120:
            done = True

        # Graykonvertion of the observation image
        #observation = cv.cvtColor(observation, cv.COLOR_BGR2GRAY)
        
        # Image preprocessing
        #vec_image = vector_space.image_preprocessing(observation)
        
        #print('REWARD: %s' % reward)
        
        #cv.imshow('vec img', vec_image)
        
        #break       # TODO: DELTE LATER
        if cv.waitKey(25) & 0xFF == ord('q'):
            cv.destroyAllWindows()
            break
    
    # Training after each episode
    for i in range(0,10):
        if agent.train:
            total_loss += agent.train_replay()
            #total_loss = agent.train_replay()
            #if loss != None:
            #    total_loss = loss
    
    
    # AFTER this Episode & the environment returns True for the done variable
    # updating the target DDQN
    agent.update_target_model()
    
    # calculating the average loss in this episode
    #episode_loss = total_loss.copy() #/ frames
    total_loss = total_loss / 512 * 10
    episode_cte = total_cte / frames
    episode_reward = total_reward / frames
    
    # updating the time parameter
    total_time = abs(total_time - time.time())
    
    # appending the collected data
    average_cte.append(episode_cte)
    average_speed.append(total_speed / frames)
    measured_time.append(total_time)
    average_rewards.append(episode_reward)
    average_loss.append(total_loss)
    collected_max_cte.append(max_cte)
    collected_lap_time.append(lap_time) 
    
    # Testing of saving the plot
    #if episode % 3 == 0 and episode != 0:
    #    save_plot(colleted_rewards, 'Reward', episode)
    #    cv.destroyAllWindows()
    #    break
    
    # Print episode information    
    print('EPISODE: %s | TIME: %s s | REWARD: %s | FRAMES: %s | QMAX: %s | EPSILON: %s | CTE: %s | LOSS: %s' % (episode, total_time, episode_reward, frames, str(agent.max_Q), agent.epsilon, episode_cte, total_loss))    
    
    # update the epsilon after each episode
    # Epsilion decay over time/amount of trainings
    if agent.epsilon >= agent.epsilon_min:
        #self.epsilon *= self.epsilon_decay
        #self.epsilon -= (self.initial_epsilon - self.epsilon_min) / self.explore
        #self.epsilon -= 0.00025
        agent.epsilon -= 0.00025 # before 0.00035
    
    if episode % save_state == 0 and episode != 0:
        print('saving after %s episodes' % save_state)
        # saving the model
        agent.save_model('C:\\Users\\studwilksa2535\\Desktop\\DonkeyCarAI\\models\\model_episode_%s.h5' % episode)        
        
        # Saving the data as plots
        save_plot(average_rewards, 'Reward', episode)
        save_plot(average_cte, 'Average cte', episode)
        save_plot(collected_max_cte, 'Max cte', episode)
        save_plot(measured_time, 'Time', episode)
        save_plot(average_loss, 'Loss', episode)
        save_plot(collected_lap_time, 'Lap Time', episode)
        
        # Saving the collected data
        save_data_as_dataframe(episode, average_rewards, average_loss, measured_time, collected_max_cte, average_cte, collected_lap_time)
    
    # closing all cv windows
    cv.destroyAllWindows()

# closing all cv windows
cv.destroyAllWindows()
# Close the enviroment
env.close()

INFO:gym_donkeycar.core.client:connecting to localhost:9091 
C:\Users\studwilksa2535\AppData\Roaming\Python\Python38\site-packages\gym\spaces\box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


starting DonkeyGym env
Setting default: start_delay 5.0
Setting default: max_cte 5.0
Setting default: frame_skip 1
Setting default: cam_resolution (120, 160, 3)
Setting default: log_level 20
Setting default: host localhost
Setting default: port 9091
loading scene generated_road


INFO:gym_donkeycar.envs.donkey_sim:on need car config
INFO:gym_donkeycar.envs.donkey_sim:sending car config.
INFO:gym_donkeycar.envs.donkey_sim:done sending cam config. {'img_w': '320', 'img_h': '200', 'img_d': '1', 'img_enc': 'PNG', 'fov': '90', 'fish_eye_x': '0.0', 'fish_eye_y': '0.0', 'offset_x': '0.0', 'offset_y': '0.0', 'offset_z': '0.0', 'rot_x': '0'}


REWARD: 9.210341021825661	CTE: 8.863837e-05
REWARD: 9.284222914197018	CTE: 3.301023e-05
REWARD: 9.356751290974682	CTE: 0.0001291196
REWARD: 9.427715009246352	CTE: 0.0007833801
REWARD: 9.504050403030165	CTE: 0.0020976
REWARD: 9.56941074867719	CTE: 0.0009010996
REWARD: 9.639177169551068	CTE: 0.0005514593
REWARD: 9.71110660422411	CTE: 0.001960902
REWARD: 9.77504302139099	CTE: 0.003498071
REWARD: 9.843558056039985	CTE: 0.002001795
REWARD: 9.909759652212319	CTE: 0.001290879
REWARD: 9.977663205729451	CTE: 0.004292198
REWARD: 10.025751403066964	CTE: 0.002780295
REWARD: 10.093345396468045	CTE: 0.0005654793
REWARD: 10.159306336263734	CTE: 0.002897453
REWARD: 10.223519104730007	CTE: 0.006881045
REWARD: 10.283726174355904	CTE: 0.01205041
REWARD: 10.346644708974802	CTE: 0.01330014
REWARD: 10.409576443943191	CTE: 0.01178355
REWARD: 10.474720795824249	CTE: 0.008813174
REWARD: 10.536804355320964	CTE: 0.007384742
REWARD: 10.571423952228399	CTE: 0.001411784
REWARD: 10.610414026711966	CTE: 0.004154702
R

REWARD: -0.6399088500479997	CTE: 1.701136
REWARD: -0.6948162124709998	CTE: 1.721487
REWARD: -0.8031990603399999	CTE: 1.754244
REWARD: -0.9177449102249997	CTE: 1.800325
REWARD: -1.0496415323272004	CTE: 1.850489
REWARD: -1.1593172797329996	CTE: 1.888685
REWARD: -1.2438540639040006	CTE: 1.925666
REWARD: -1.3423820448319996	CTE: 1.962672
REWARD: -3.5255732751096005	CTE: 2.019277
REWARD: -3.7501250773068002	CTE: 2.076117
REWARD: -3.9152301086879993	CTE: 2.112866
REWARD: -4.014599967201599	CTE: 2.143316
REWARD: -4.1446725155304005	CTE: 2.176349
REWARD: -4.218942557691601	CTE: 2.199914
REWARD: -4.287205190501599	CTE: 2.219511
REWARD: -4.459697253788001	CTE: 2.256015
REWARD: -4.600333555353599	CTE: 2.297408
REWARD: -4.761168502244999	CTE: 2.333525
REWARD: -4.8478145759343985	CTE: 2.359282
REWARD: -4.944754333940001	CTE: 2.389021
REWARD: -5.068688666316401	CTE: 2.423083
REWARD: -5.144526995628	CTE: 2.444303
REWARD: -5.209707868710002	CTE: 2.465765
REWARD: -5.298512375482999	CTE: 2.486385
REWARD

...

EPISODE: 1522 | TIME: 74.00159096717834 s | REWARD: 3.462674139382034 | FRAMES: 532 | QMAX: 284.15314 | EPSILON: 0.019750000000082396 | CTE: -1.10235099641712 | LOSS: 15.140876024961472
EPISODE: 1523 | TIME: 90.04874086380005 s | REWARD: 7.7586804658210955 | FRAMES: 651 | QMAX: 457.13892 | EPSILON: 0.019750000000082396 | CTE: -0.18310255472749615 | LOSS: 14.944759532809258
EPISODE: 1524 | TIME: 274.69662737846375 s | REWARD: -9.832732647897043 | FRAMES: 2106 | QMAX: 225.98697 | EPSILON: 0.019750000000082396 | CTE: -3.02728105656043 | LOSS: 17.259066253900528
EPISODE: 1525 | TIME: 192.3908290863037 s | REWARD: -14.418284662081769 | FRAMES: 1465 | QMAX: 413.91092 | EPSILON: 0.019750000000082396 | CTE: -3.6510919612440267 | LOSS: 18.581156134605408
EPISODE: 1526 | TIME: 146.37339901924133 s | REWARD: -4.191401272705844 | FRAMES: 1094 | QMAX: 166.89516 | EPSILON: 0.019750000000082396 | CTE: -2.106746601912246 | LOSS: 17.190675288438797
EPISODE: 1527 | TIME: 77.05000758171082 s | REWARD: 9.

EPISODE: 1566 | TIME: 360.67008805274963 s | REWARD: 8.399491518348336 | FRAMES: 2798 | QMAX: 326.41437 | EPSILON: 0.019750000000082396 | CTE: 0.1526406476460684 | LOSS: 12.210686430335045
EPISODE: 1567 | TIME: 374.04084300994873 s | REWARD: 2.2091823968223685 | FRAMES: 2895 | QMAX: 135.87361 | EPSILON: 0.019750000000082396 | CTE: -0.9518370047906742 | LOSS: 13.403357714414597
EPISODE: 1568 | TIME: 80.85543370246887 s | REWARD: 9.466270803047127 | FRAMES: 581 | QMAX: 322.44785 | EPSILON: 0.019750000000082396 | CTE: 0.13669152236678142 | LOSS: 12.46978722512722
EPISODE: 1569 | TIME: 176.98340249061584 s | REWARD: 9.019726850383439 | FRAMES: 1345 | QMAX: 243.2976 | EPSILON: 0.019750000000082396 | CTE: 0.07853458272014864 | LOSS: 11.781371384859085
EPISODE: 1570 | TIME: 327.674866437912 s | REWARD: 3.290122352195926 | FRAMES: 2534 | QMAX: 305.52765 | EPSILON: 0.019750000000082396 | CTE: -0.770751820559956 | LOSS: 13.38224209845066
EPISODE: 1571 | TIME: 48.86416530609131 s | REWARD: 4.5563

EPISODE: 1610 | TIME: 30.59745693206787 s | REWARD: 5.732512959617529 | FRAMES: 185 | QMAX: 402.48233 | EPSILON: 0.019750000000082396 | CTE: -0.29891663308432437 | LOSS: 10.55747926235199
EPISODE: 1611 | TIME: 118.62944102287292 s | REWARD: 2.5200886227185393 | FRAMES: 881 | QMAX: 340.63135 | EPSILON: 0.019750000000082396 | CTE: -1.0799426804598244 | LOSS: 11.687383875250816
EPISODE: 1612 | TIME: 79.57186484336853 s | REWARD: 3.5184179400223408 | FRAMES: 578 | QMAX: 219.38802 | EPSILON: 0.019750000000082396 | CTE: -0.9433410376122662 | LOSS: 11.314336359500885
EPISODE: 1613 | TIME: 82.10420513153076 s | REWARD: 8.381283622353719 | FRAMES: 590 | QMAX: 345.33765 | EPSILON: 0.019750000000082396 | CTE: -0.31789784776203384 | LOSS: 10.043972730636597
EPISODE: 1614 | TIME: 172.63739585876465 s | REWARD: 7.678599074696699 | FRAMES: 1308 | QMAX: 366.23294 | EPSILON: 0.019750000000082396 | CTE: -0.5598891267648695 | LOSS: 12.477053105831146
EPISODE: 1615 | TIME: 83.20036768913269 s | REWARD: 9.

EPISODE: 1654 | TIME: 267.7020728588104 s | REWARD: -4.844552349543167 | FRAMES: 2054 | QMAX: 282.71384 | EPSILON: 0.019750000000082396 | CTE: -2.215990817284506 | LOSS: 12.870841771364212
EPISODE: 1655 | TIME: 84.1718978881836 s | REWARD: 10.10627998020435 | FRAMES: 610 | QMAX: 226.16656 | EPSILON: 0.019750000000082396 | CTE: -0.02057432524757374 | LOSS: 11.464385241270065
EPISODE: 1656 | TIME: 90.07489371299744 s | REWARD: 7.005005243683587 | FRAMES: 658 | QMAX: 430.3349 | EPSILON: 0.019750000000082396 | CTE: 0.37561228582993933 | LOSS: 10.79614870250225
EPISODE: 1657 | TIME: 84.0112292766571 s | REWARD: 5.158448559247646 | FRAMES: 609 | QMAX: 282.74918 | EPSILON: 0.019750000000082396 | CTE: 0.7349344237779966 | LOSS: 11.711008995771408
EPISODE: 1658 | TIME: 83.36331081390381 s | REWARD: 5.599466838612156 | FRAMES: 600 | QMAX: 275.36023 | EPSILON: 0.019750000000082396 | CTE: 0.6821968391558865 | LOSS: 10.684746503829956
EPISODE: 1659 | TIME: 161.06229853630066 s | REWARD: 3.273349999

KeyboardInterrupt: 